<a href="https://colab.research.google.com/github/Venkatpandey/DataScience_ML/blob/main/ml/Hyper_parameter_Tuning_(GridSearchCV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
from sklearn.datasets import load_digits
digits = load_digits()
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [12]:
print(digits.target_names)

[0 1 2 3 4 5 6 7 8 9]


In [14]:
print(digits.data.size)

115008


In [5]:
df = pd.DataFrame(digits.data, columns = digits.feature_names)
df.head()

,pixel_0_0,pixel_0_1,pixel_0_2,pixel_0_3,pixel_0_4,pixel_0_5,pixel_0_6,pixel_0_7,pixel_1_0,pixel_1_1,pixel_1_2,pixel_1_3,pixel_1_4,pixel_1_5,pixel_1_6,pixel_1_7,pixel_2_0,pixel_2_1,pixel_2_2,pixel_2_3,pixel_2_4,pixel_2_5,pixel_2_6,pixel_2_7,pixel_3_0,pixel_3_1,pixel_3_2,pixel_3_3,pixel_3_4,pixel_3_5,pixel_3_6,pixel_3_7,pixel_4_0,pixel_4_1,pixel_4_2,pixel_4_3,pixel_4_4,pixel_4_5,pixel_4_6,pixel_4_7,pixel_5_0,pixel_5_1,pixel_5_2,pixel_5_3,pixel_5_4,pixel_5_5,pixel_5_6,pixel_5_7,pixel_6_0,pixel_6_1,pixel_6_2,pixel_6_3,pixel_6_4,pixel_6_5,pixel_6_6,pixel_6_7,pixel_7_0,pixel_7_1,pixel_7_2,pixel_7_3,pixel_7_4,pixel_7_5,pixel_7_6,pixel_7_7
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,13.0,15.0,10.0,15.0,5.0,0.0,0.0,3.0,15.0,2.0,0.0,11.0,8.0,0.0,0.0,4.0,12.0,0.0,0.0,8.0,8.0,0.0,0.0,5.0,8.0,0.0,0.0,9.0,8.0,0.0,0.0,4.0,11.0,0.0,1.0,12.0,7.0,0.0,0.0,2.0,14.0,5.0,10.0,12.0,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,9.0,0.0,0.0,0.0,0.0,3.0,15.0,16.0,6.0,0.0,0.0,0.0,7.0,15.0,16.0,16.0,2.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,3.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,1.0,16.0,16.0,6.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,3.0,16.0,15.0,14.0,0.0,0.0,0.0,0.0,8.0,13.0,8.0,16.0,0.0,0.0,0.0,0.0,1.0,6.0,15.0,11.0,0.0,0.0,0.0,1.0,8.0,13.0,15.0,1.0,0.0,0.0,0.0,9.0,16.0,16.0,5.0,0.0,0.0,0.0,0.0,3.0,13.0,16.0,16.0,11.0,5.0,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,13.0,6.0,15.0,4.0,0.0,0.0,0.0,2.0,1.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,2.0,15.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,8.0,0.0,0.0,0.0,8.0,4.0,5.0,14.0,9.0,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,6.0,2.0,2.0,0.0,0.0,0.0,7.0,15.0,0.0,9.0,8.0,0.0,0.0,5.0,16.0,10.0,0.0,16.0,6.0,0.0,0.0,4.0,15.0,16.0,13.0,16.0,1.0,0.0,0.0,0.0,0.0,3.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2)
print(X_train.size)

91968


In [16]:
svc_model = svm.SVC(kernel='rbf', C=30, gamma='auto')
svc_model.fit(X_train, y_train)
svc_model.score(X_test, y_test)

0.5444444444444444

In [17]:
from sklearn.model_selection import cross_val_score


In [18]:
cross_val_score(svm.SVC(kernel='rbf', C=30, gamma='auto'), digits.data, digits.target, cv=5)

array([0.45277778, 0.46944444, 0.47910864, 0.47910864, 0.50139276])

In [19]:
cross_val_score(svm.SVC(kernel='rbf', C=10, gamma='auto'), digits.data, digits.target, cv=5)

array([0.45277778, 0.46944444, 0.47910864, 0.47910864, 0.50139276])

In [20]:
cross_val_score(svm.SVC(kernel='linear', C=20, gamma='auto'), digits.data, digits.target, cv=5)

array([0.96388889, 0.91944444, 0.96657382, 0.9637883 , 0.92479109])

In [22]:
# using for loop for multiple params
import numpy as np
kernel = ['rbf', 'linear']
C = [1,10,15,20,25,30]
avg_score = {}
for k in kernel:
  for cv in C:
    cv_score = cross_val_score(svm.SVC(kernel=k, C=cv, gamma='auto'), digits.data, digits.target, cv=5)
    avg_score[k + '_' + str(cv)] = np.average(cv_score)
print(avg_score)

{'rbf_1': 0.448545341999381, 'rbf_10': 0.47636645001547506, 'rbf_15': 0.47636645001547506, 'rbf_20': 0.47636645001547506, 'rbf_25': 0.47636645001547506, 'rbf_30': 0.47636645001547506, 'linear_1': 0.9476973073351903, 'linear_10': 0.9476973073351903, 'linear_15': 0.9476973073351903, 'linear_20': 0.9476973073351903, 'linear_25': 0.9476973073351903, 'linear_30': 0.9476973073351903}


In [23]:
# Using GridSearchCV
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C' : [1,10,15,20,25,30],
    'kernel' : ['rbf', 'linear']
}, cv = 5, return_train_score=False)

clf.fit(digits.data, digits.target)
clf.cv_results_

{'mean_fit_time': array([0.36505866, 0.02591152, 0.28338547, 0.02618837, 0.27820978,
        0.02619467, 0.28497114, 0.02740097, 0.28321948, 0.02646513,
        0.28334894, 0.02644825]),
 'mean_score_time': array([0.11297073, 0.00640664, 0.06345911, 0.00638433, 0.06588612,
        0.00663767, 0.06365089, 0.00673695, 0.06265321, 0.00690265,
        0.06329412, 0.00634217]),
 'mean_test_score': array([0.44854534, 0.94769731, 0.47636645, 0.94769731, 0.47636645,
        0.94769731, 0.47636645, 0.94769731, 0.47636645, 0.94769731,
        0.47636645, 0.94769731]),
 'param_C': masked_array(data=[1, 1, 10, 10, 15, 15, 20, 20, 25, 25, 30, 30],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear',
                    'rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[F

In [26]:
cv_result = pd.DataFrame(clf.cv_results_)
print(cv_result[['param_C', 'param_kernel', 'mean_test_score']])

   param_C param_kernel  mean_test_score
0        1          rbf         0.448545
1        1       linear         0.947697
2       10          rbf         0.476366
3       10       linear         0.947697
4       15          rbf         0.476366
5       15       linear         0.947697
6       20          rbf         0.476366
7       20       linear         0.947697
8       25          rbf         0.476366
9       25       linear         0.947697
10      30          rbf         0.476366
11      30       linear         0.947697


In [27]:
clf.best_score_

0.9476973073351903

In [28]:
clf.best_params_

{'C': 1, 'kernel': 'linear'}

In [29]:
# Randomised SerachCV
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
    'C' : [1,10,15,20,25,30],
    'kernel' : ['rbf', 'linear']
}, cv = 5, return_train_score=False, n_iter=2)

In [32]:
rs.fit(digits.data, digits.target)
print(pd.DataFrame(rs.cv_results_)[['param_C', 'param_kernel', 'mean_test_score']])

  param_C param_kernel  mean_test_score
0       1          rbf         0.448545
1      25       linear         0.947697


In [38]:
model_param = {
    'svm' : {
        'model' : svm.SVC(gamma='auto'),
        'params' : {
            'C' : [1,10,20,30],
            'kernel' : ['rbf', 'linear']
        }
    },
    'random_forest' : {
      'model' : RandomForestClassifier(),
      'params' : {
          'n_estimators' : [1,5,10]
      }
    },
    'logistic_regressions' : {
        'model' : LogisticRegression(solver='liblinear', multi_class='auto'),
        'params' : {
            'C' : [1,5,10]
        }
    },
    'gaussian' : {
        'model' : GaussianNB(),
        'params' : {
            'var_smoothing' : [1.0, 2.5]
        }
    },
    'multinomial' : {
        'model' : MultinomialNB(),
        'params' : {
            'alpha' : [1.5, 2.5]
        }
    },
    'decisionTree' : {
        'model' : DecisionTreeClassifier(),
        'params' : {
            'criterion' : ['gini', 'entropy'],
            'max_depth' : [10]

        }
    }
}

In [39]:
scores = []

for mn,mp in model_param.items():
  clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
  clf.fit(digits.data, digits.target)
  scores.append({
      'model' : mn,
      'best_score' : clf.best_score_,
      'best_params' :clf.best_params_
  })

In [41]:
print(pd.DataFrame(scores, columns=('model', 'best_score', 'best_params')))

                  model  best_score                                best_params
0                   svm    0.947697               {'C': 1, 'kernel': 'linear'}
1         random_forest    0.903208                       {'n_estimators': 10}
2  logistic_regressions    0.922114                                   {'C': 1}
3              gaussian    0.882030                     {'var_smoothing': 1.0}
4           multinomial    0.872021                             {'alpha': 2.5}
5          decisionTree    0.808584  {'criterion': 'entropy', 'max_depth': 10}
